## import library
- http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
- http://learningtensorflow.com/index.html
- http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

# 0. Tensorflow basic

- **tf.constant(), tf.Variable()**: 그래프의 객체
- **tf.Session()/ tf.InteractiveSession()**: 그래프를 시작
- **sess.run(c)**: 텐서 'c'를 계산 -> sess.close() 해야함
- **with 절**: tf.Session()은 with 절과 사용, with 절에서 sess.close() 생략가능
- **c.eval()**: sess.run(c)와 같음

### - **tf.variable_scope()**
- 모델에서 필요한 변수들을 관리하는 클래스(변수 범위 만들기)
- tf.get_variable(): 직접호출 없이 변수를 가져오거나 생성(initializer 사용)
- reuse = False: '현재 variable scope 이름 + 제공된 name'이 없으면 생성
- reuse = True: '현재 variable scope 이름 + 제공된 name'이 있으면 반환
- reuse = AUTO_REUSE: 둘다

# 3. Long Sequence RNN(automatic)

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

sample = "Hello my name is hyejin"
idx2char = list(set(sample))  # index -> char
# 중복 제거
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
# character별 숫자 딕셔너리
print(idx2char)
print(char2idx)

['h', 'y', 'm', 'l', 'e', 'i', ' ', 'a', 'o', 'n', 'H', 's', 'j']
{'h': 0, 'y': 1, 'm': 2, 'l': 3, 'e': 4, 'i': 5, ' ': 6, 'a': 7, 'o': 8, 'n': 9, 'H': 10, 's': 11, 'j': 12}


### Set parameters

In [10]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)

batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1
print(dic_size, rnn_hidden_size, num_classes)
print(sequence_length)

13 13 13
22


### One-hot encoding

In [11]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
                            #batch_size = 1

# flatten the data (ignore batches for now). No effect if the batch size is 1
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
print(X)
print(Y)
print(x_one_hot)

Tensor("Placeholder:0", shape=(?, 22), dtype=int32)
Tensor("Placeholder_1:0", shape=(?, 22), dtype=int32)
Tensor("one_hot:0", shape=(?, 22, 13), dtype=float32)


### Create cell

In [12]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

### Train RNN

In [13]:
# reshape out for sequence_loss
print(outputs.shape)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
print(outputs.shape)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

(1, 22, 13)
(1, 22, 13)


In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.5710404 Prediction: en mmmmmmmmmmmm miiiin
1 loss: 2.4674807 Prediction: e             n     nn
2 loss: 2.340106 Prediction: e          e in iyijin
3 loss: 2.270538 Prediction: e    m  m             
4 loss: 2.2247298 Prediction: e               iyeiin
5 loss: 2.158218 Prediction: e                   in
6 loss: 2.1276743 Prediction: e                     
7 loss: 2.0745318 Prediction: eooo m                
8 loss: 2.021412 Prediction: eooo m                
9 loss: 1.9701705 Prediction: eooo m               n
10 loss: 1.9194913 Prediction: eooo m           y  in
11 loss: 1.8725915 Prediction: elll m        n hy  in
12 loss: 1.823976 Prediction: elll m       nn hy jin
13 loss: 1.7782437 Prediction: elll m  n me nn hyejin
14 loss: 1.7366173 Prediction: elll my n me in hyejin
15 loss: 1.7046306 Prediction: elll my nmme in hyejin
16 loss: 1.6776389 Prediction: elll my nmme in hyejin
17 loss: 1.6515915 Prediction: elll my nmme in hyejin
18 loss: 1.6263705 Prediction: elll my nmme